# TuoLing: An simple example for fine-tune ChatGLM
Developed by Ziang Leng 冷子昂, Qiyuan Chen 陈启源 and Cheng Li 李鲁鲁.
This is a simple script based on the project of luotuo-silk-road.git

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LC1332/Luotuo-Chinese-LLM/blob/main/notebook/TuoLing_evaluation_code.ipynb)

In [ ]:
!git clone https://github.com/LC1332/luotuo-silk-road.git ./luotuo_silk_road
!cd luotuo_silk_road/TuoLing && pip install -r requirements.txt 

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

torch.set_default_tensor_type(torch.cuda.HalfTensor)

model = AutoModel.from_pretrained (
    "THUDM/chatglm-6b", 
    trust_remote_code=True, 
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained (
    "THUDM/chatglm-6b", 
    trust_remote_code=True
)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "./luotuo_silk_road/TuoLing/output/chatglm-lora.pt"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=True,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

model.load_state_dict(torch.load(peft_path), strict=False)

torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
from luotuo_silk_road.TuoLing.cover_alpaca2jsonl import format_example


def evaluate(instruction, input=None):
    with torch.no_grad():
        feature = format_example(
            {"instruction": f"{instruction}", "output": "", "input": ""}
        )
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        out = model.generate(input_ids=input_ids, max_length=150, temperature=0)
        answer = tokenizer.decode(out[0])
        print(answer)

In [ ]:
evaluate(input("Instruction: "))

In [ ]:
evaluate(input("Instruction: "))